In [ ]:
import os
import json
import re
from langchain_groq import ChatGroq
from langchain.agents import initialize_agent, AgentType, Tool
from langchain_community.utilities import SerpAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate

os.environ["GROQ_API_KEY"] = "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"
os.environ["SERPAPI_API_KEY"] = "bd0d819e093e6c8acf32d6aaab551208bb2f15b199966fa215daa790ba73c33d"

llm = ChatGroq(api_key=os.environ["GROQ_API_KEY"], model_name="llama3-70b-8192")

search_tool = SerpAPIWrapper()
python_tool = PythonREPLTool()

response_schemas = [
    ResponseSchema(name="title", description="Title of the research topic"),
    ResponseSchema(name="summary", description="Short summary in 2-3 lines"),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

prompt_template = PromptTemplate(
    template="""You are a research assistant. Provide a short summary about: {topic}
Return the response in valid JSON format as shown below. Ensure proper JSON syntax with correct commas and quotes:
```json
{format_instructions}
```""",
    input_variables=["topic"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

def structured_summary(topic: str) -> dict:
    prompt = prompt_template.format(topic=topic)
    try:
        response = llm.invoke(prompt)
        raw_content = response.content.strip()
        print("Raw LLM Response:", raw_content)  

        json_match = re.search(r'```json\n([\s\S]*?)\n```', raw_content)
        if json_match:
            json_str = json_match.group(1)
        else:
            json_str = raw_content

        parsed = output_parser.parse(json_str)
        return parsed
    except json.JSONDecodeError as e:
        print(f"JSON Parsing Error: {e}")
        return {"title": "Error", "summary": f"Failed to parse LLM response: {str(e)}"}
    except Exception as e:
        print(f"Unexpected Error: {e}")
        return {"title": "Error", "summary": f"Unexpected error: {str(e)}"}

tools = [
    Tool(
        name="Web Search",
        func=search_tool.run,
        description="Use this tool to search for recent research using SerpAPI"
    ),
    Tool(
        name="Python REPL",
        func=python_tool.run,
        description="Useful for calculations and logic"
    ),
    Tool(
        name="Structured Summary",
        func=structured_summary,
        description="Summarize a research topic in 2–3 lines with title"
    )
]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

query = """
1. Find the latest research in LLMs and summarize it using the Structured Summary tool.
2. Calculate how many tokens are needed for 3 million words using the Python REPL tool, assuming 1 word = 1.33 tokens.
"""
try:
    result = agent.invoke(query)
    print("\n🧠 Final Answer:\n", result)
except Exception as e:
    print(f"Agent Execution Error: {e}")
    print("\n🧠 Final Answer:\n", {"error": f"Agent failed to execute: {str(e)}"})



> Entering new AgentExecutor chain...
Question: Find the latest research in LLMs and summarize it, and calculate tokens needed for 3 million words.

Thought: Let's break this down into two tasks. First, I'll find the latest research in LLMs and summarize it. Then, I'll calculate the tokens needed for 3 million words.

Action:
```
{
  "action": "Web Search",
  "action_input": "latest research in LLMs"
}
```

Observation: ['The latest in LLM research with a hand-curated, topic-organized list of over 200 research papers from 2025.', 'With advancements in LLMs across industries, Booth faculty are key figures leading broader application of language models. We highlight a few recent ...', 'Beyond the Experiment: What Academic Research Reveals About Cognitive Stretching in LLMs. https://medium.com/@ResponseLab/beyond-the ...', 'This article explores the future of large language models by delving into developments like self-training, fact-checking, and sparse ...', 'The LLMs Research Paper i

Python REPL can execute arbitrary code. Use with caution.


Thought: Now that I have summarized the latest research in LLMs, I'll calculate the tokens needed for 3 million words.

Action:
```
{
  "action": "Python REPL",
  "action_input": "3 * 10**6 * 1.33"
}
```

Observation: 
Thought:Action:
```
{
  "action": "Python REPL",
  "action_input": "print(3 * 10**6 * 1.33)"
}
```

Observation: 3990000.0

Thought:Thought: I have the summary of the latest research in LLMs and the calculation of tokens needed for 3 million words. I'll provide the final answer.

Action:
```
{
  "action": "Final Answer",
  "action_input": "The latest research in LLMs focuses on scaling up model sizes, leading to significant improvements in natural language understanding and generation capabilities. Recent advancements include the use of massive datasets, novel architectures, and efficient training algorithms. For 3 million words, approximately 3,990,000 tokens are needed, assuming 1 word = 1.33 tokens."
}
```

> Finished chain.

🧠 Final Answer:
 {'input': '\n1. Find the 